In [1]:
## Load environment variables

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
## Define the LLM Model and the Embedding model

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
llm=ChatOpenAI(model="gpt-4o-mini",temperature=0)
embeddings=OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1536)

In [2]:
## Load the documents

from langchain_community.document_loaders import Docx2txtLoader
loader_bsw = Docx2txtLoader("BSW SOM 0.1.docx")
loader_npac = Docx2txtLoader("NPAC COREX SOM v2.0.docx")
data_bsw = loader_bsw.load()
data_npac = loader_npac.load()

In [8]:
import re
from langchain_openai import ChatOpenAI
from typing import Any, List
from langchain_text_splitters import TextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document

In [3]:
## Define a custom splitter

class GPTSplitter(TextSplitter):
    def __init__(self, model_name: str = "gpt-4o-mini", **kwargs: Any) -> None:
        super().__init__(**kwargs)
        self.model = ChatOpenAI(model=model_name)

        self.prompt = ChatPromptTemplate.from_template(
            """You are an expert in analyzing and structuring technical documentation, particularly Service Operation Manuals (SOMs). Your task is to divide the given text into coherent, meaningful chunks that preserve both the document's structure and its detailed content. Each chunk should encapsulate a complete section or subsection, including all relevant details, explanations, and bullet points.

                Follow these guidelines:
                1. Preserve the hierarchical structure (e.g., main sections, subsections) and include all content under each header.
                2. Keep related information together (e.g., lists, tables, contact information, detailed explanations).
                3. Ensure each chunk is self-contained and includes all necessary context and details.
                4. Aim for chunks that cover complete sections or logical groupings of subsections.
                5. Include all text content, not just headers or bullet points.
                6. Maintain the original formatting and structure within each chunk.

                Wrap each chunk in <<<>>> markers.

                Example:
                <<<3. Centiq Managed Service
                3.1. Introduction
                • Centiq provides comprehensive managed services for SAP environments
                • Our team of experts ensures 24/7 monitoring and support
                • We offer proactive maintenance and optimization
                [Include all bullet points and any additional explanatory text here]

                3.2. Managed Service Description
                3.2.1. Service Operations
                • 24/7 monitoring of SAP systems and infrastructure
                • Incident management and resolution
                • Performance tuning and optimization
                [Include all bullet points and detailed descriptions of each service operation]>>>

                Now, process the following text from the Service Operation Manual, ensuring to include ALL content and details:

                {text}"""
        )
        self.output_parser = StrOutputParser()
        self.chain = (
            {"text": RunnablePassthrough()}
            | self.prompt
            | self.model
            | self.output_parser
        )

    # def split_text(self, text: str) -> List[str]:
    #     response = self.chain.invoke({"text": text})
    #     # Use regex to split properly by <<< and >>> markers
    #     chunks = re.findall(r'<<<(.*?)>>>', response, re.DOTALL)
    #     return [chunk.strip() for chunk in chunks]
    
    def split_text(self, text: str) -> List[Document]:
      chunks = self.chain.invoke(text).split('<<<')[1:]  # Split the result and remove the first empty element
      return [Document(page_content=chunk.strip('>>>').strip()) for chunk in chunks]

In [4]:
## Split the documents using the custom splitter

gpt_splitter = GPTSplitter()
bsw_docs = gpt_splitter.split_text(data_bsw)
npac_docs = gpt_splitter.split_text(data_npac)

In [5]:
for doc in bsw_docs:
    doc.metadata["source"] = "BSW"

for doc in npac_docs:
    doc.metadata["source"] = "NPAC"

In [6]:
all_chunks = bsw_docs + npac_docs

In [4]:
from langchain_chroma import Chroma

In [7]:
# Create Chroma vector store

Chroma.from_documents(
    documents=all_chunks,
    embedding=embeddings,
    persist_directory="./SOM"
)

In [5]:
## loading the vector database from local db

vectordb=Chroma(persist_directory="SOM",embedding_function=embeddings)

In [6]:
retriever=vectordb.as_retriever(search_kwargs={"k": 10})

In [9]:
## Generating the multiquery retriever

from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
import re


QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an expert query rewriter specializing in SAP Basis managed services. Your task is to take an original query and generate at least upto different versions of the same query. These variations should:

                1. Maintain the core intent of the original query
                2. Use different phrasings, synonyms, and technical terms
                3. Vary in complexity, from simple to more detailed
                4. Include relevant SAP Basis-specific terminology where appropriate
                5. Consider different aspects or perspectives of the original question
                6. Utilize common SAP Basis synonyms and alternative terms where possible
                7. Consider using alternate terms where appropriate, e.g system refresh for system copy, etc

                Please provide upto 3 rewritten versions of this query, ensuring that each version is distinct and adds value to the search process. Label each rewritten query clearly.

                Provide these alternative question like this:
                    <<question1>>
                    <<question2>>
                Only provide the query, no numbering.

                Remember to consider various user roles (e.g., SAP Basis administrators, support staff, managers) and their potential ways of asking the same question.

                Original question: {question}""",
)


def split_and_clean_text(input_text):
    return [item for item in re.split(r"<<|>>", input_text) if item.strip()]

multiquery_chain = (
    QUERY_PROMPT | llm | StrOutputParser() | RunnableLambda(split_and_clean_text)
)

In [10]:
## Creating a function for returning unique documents(remove duplicate docs)

def flatten_and_unique_documents(documents):
    flattened_docs = [doc for sublist in documents for doc in sublist]

    unique_docs = []
    unique_contents = set()
    for doc in flattened_docs:
        if doc.page_content not in unique_contents:
            unique_docs.append(doc)
            unique_contents.add(doc.page_content)

    return unique_docs

def find_rel_docs(query):
    list_of_questions = multiquery_chain.invoke(query)
    docs = [retriever.invoke(q) for q in list_of_questions]
    final_docs=flatten_and_unique_documents(documents=docs)
    return final_docs

In [11]:
retrieve_chain=multiquery_chain | RunnableLambda(find_rel_docs)

In [12]:
## Generate an answer

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub

answer_template = """You are an AI assistant tasked with answering questions based on the provided documents. Your goal is to generate accurate and informative answers using only the information contained in these documents.

When answering, please follow these guidelines:
1. Analyze the given question and the provided context documents carefully.
2. Formulate your answer in a clear, concise, and structured manner.
3. Present your answer in a point-wise format, using numbered points for main ideas or steps.
4. Use sub-points (a, b, c) if necessary to provide additional details under a main point.
5. Ensure each point is directly relevant to the question and supported by the context documents.
6. If the information in the documents is insufficient to fully answer the question, state this clearly and provide any partial information that is available.
7. Do not contradict yourself or provide inconsistent information.

Question: {question} 

Context: {context}

Answer:"""

answer_prompt = ChatPromptTemplate.from_template(answer_template)

llm=ChatOpenAI(model="gpt-4o-mini",temperature=0.1)

def format_docs(relevant_docs):
    return "\n\n".join(doc.page_content for doc in relevant_docs)



answer_chain = (
    {"context":retrieve_chain | format_docs, "question": RunnablePassthrough()} # type: ignore
    | answer_prompt
    | llm
    | StrOutputParser()
)

In [13]:
from typing import List

from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        list_of_questions: list of questions
        generation: LLM generation
        documents: list of documents
    """

    question: str
    list_of_questions: List[str]
    answer: str
    documents: List[str]

In [14]:
from langchain.schema import Document
import time


def query_rewriter(state):
    """
    Generate 5 alternate queries

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, list of questions
    """
    print("---GENERATING ALTERNATE QUERIES---")
    question = state["question"]

    # Generating questions
    list_of_questions = multiquery_chain.invoke(question)
    for query in list_of_questions:
        print(f"- {query}")
        time.sleep(0.5)
    return {"list_of_questions": list_of_questions}


# def retrieve_docs(state):
#     """
#     Return unique documents that matches the context of the question asked

#     Args:
#         state (dict): The current graph state

#     Returns:
#         state (dict): New key added to state, retrieved documents
#     """
#     print("---GETTING DOCUMENTS---")
#     list_of_questions = state["list_of_questions"]
#     docs = [retriever.invoke(q) for q in list_of_questions]
#     final_docs=flatten_and_unique_documents(documents=docs)
    
#     return {"documents": final_docs}


def generate(state):
    """
    Generates an answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Generates the final answer
    """

    print("---GENERATING ANSWER---")

    documents = state["documents"]
    query = state["question"]
    answer = answer_chain.invoke({"question":query})
    return {"answer":answer}

# Build Graph

In [15]:
from langgraph.graph import END, StateGraph, START
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("query_rewriter", query_rewriter)  # rewrite query
# workflow.add_node("retrieve", retrieve_docs)  # retrieve documents
workflow.add_node("generate", generate)  # generate answer

# Build graph
workflow.add_edge(START, "query_rewriter")    
#workflow.add_edge("query_rewriter", "retrieve")
#workflow.add_edge("retrieve", "generate")
workflow.add_edge("query_rewriter", "generate")
workflow.add_edge("generate",END)

# Compile
app = workflow.compile(checkpointer=memory)

In [16]:
from pprint import pprint
import uuid

def generate_session_id():
    return str(uuid.uuid4())

def chatbot():
   
    print("Welcome to the Service Operations Manual Chatbot!")
    print("Ask questions about BSW or NPAC, or type 'quit' to exit.")

    # Generate a new session ID for this conversation
    session_id = generate_session_id()
    
    while True:
        query = input("\nYour question: ").strip()
        
        if query.lower() == 'quit':
            print("Thank you for using the chatbot. Goodbye!")
            break
        
        if not query:
            print("Please enter a valid question.")
            continue
        
        print("Thinking...")
        
        try:

            # Use the session_id in the config
            config = {"configurable": {"thread_id": session_id}}

            for output in app.stream({"question":query}, config):
                for key, value in output.items():
                    # Node
                    print(f"Node '{key}':")
                    # Optional: print full state at each node
                    # pprint(value["keys"], indent=2, width=80, depth=None)
                print("\n---\n")

            # Final generation
            print(value["answer"])
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            print("Please try asking your question in a different way.")

# Run the chatbot
if __name__ == "__main__":
    chatbot()


Welcome to the Service Operations Manual Chatbot!
Ask questions about BSW or NPAC, or type 'quit' to exit.
Thinking...
---GENERATING ALTERNATE QUERIES---
- What is the total number of system copies included in the scope for BSW?
- Can you provide the count of system refreshes that fall under the BSW scope?
- How many system clones are accounted for within the BSW parameters?
Node 'query_rewriter':

---

---GENERATING ANSWER---
Node 'generate':

---

Based on the provided context documents, the information regarding the number of system copies in scope for BSW is not explicitly stated. However, I can summarize the relevant details:

1. **System Management Activities**:
   - The documents mention activities related to SAP Basis operations, which include managing landscape diagrams and client strategies, such as local client creation and copies.
   - Specifically, it notes that "local client copies" are scheduled as required, indicating that there is a process for managing system copies.
